# Project - Third Requirement

### Include packages.

In [1]:
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from scipy.stats import norm, gamma
from itertools import product
from scipy.optimize import linprog

### As usual, as first, define the environment.

In [2]:
class Environment:
    def __init__(self):
        pass
    def round(self, a_t):
        pass

In [ ]:
# Here we model the probability of buying around the mean valuation as a gaussian distribution...
# ... but its mean keeps varying as it is sampled - more into detail, from two possible distributions.
class PricingEnvironmentSingleProductMultiDistribution(Environment):
    def __init__(self, cost, dist_1_params, dist_2_params, buyers_per_round, std_val):
        self.N = len(cost)
        self.cost = cost
        self.dist_1_params = dist_1_params
        self.dist_2_params = dist_2_params
        self.buyers_per_round = buyers_per_round
        self.std_val = std_val

    def round(self, p_t, dist):
        # It is possible to choose between two distributions (possibly randomly, user's choice)...
        if dist % 2 == 0:
            mean_val = self.dist_1_params[0]
            std_val = self.dist_1_params[1]
            sample_valuation = norm.rvs(loc=mean_val, scale=std_val)
        else:
            shape = self.dist_2_params[0]
            scale = self.dist_2_params[1]
            sample_valuation = gamma.rvs(a=shape, scale=scale)
        
        # We sample the gaussian mean (where probability of buying is 50%) from the chosen distribution... 
        # ...  and get the probability of a single customer buying at price (quantile) p_t
        prob_buy = 1 - norm.cdf(p_t, loc=sample_valuation, scale=self.std_val)
        
        # Number of sales is then drawn from a binomial distribution
        num_of_sales = np.random.binomial(n=self.buyers_per_round, p=prob_buy)
        
        reward = (p_t - self.cost) * num_of_sales
        return num_of_sales, reward

In [ ]:
class PricingEnvironmentSingleProductSinusoidalMean(Environment):
    def __init__(self, cost, initial_base_mean_valuation, std_valuation, buyers_per_round, total_rounds, amplitude, frequency):
        self.cost = cost
        self.initial_base_mean_valuation = initial_base_mean_valuation
        self.std_valuation = std_valuation
        self.buyers_per_round = buyers_per_round
        self.total_rounds = total_rounds            # Needed to define the full cycle
        self.current_round = 0                      # To track time
        self.amplitude = amplitude                  # How much the mean valuation swings up and down
        self.frequency = frequency                  # How many rounds it takes to complete one full cycle

    def round(self, p_t):
        current_mean_valuation = self.initial_base_mean_valuation + self.amplitude * np.sin(2 * np.pi * self.current_round / self.frequency)

        # Probability of a single customer buying at price p_t
        # loc=current_mean_valuation means the center of the valuation distribution shifts
        prob_buy = 1 - norm.cdf(p_t, loc=current_mean_valuation, scale=self.std_valuation)

        # Ensure probability is within valid range [0, 1] due to extreme values in sine wave
        prob_buy = np.clip(prob_buy, 0, 1)

        # Number of sales is drawn from a binomial distribution
        num_of_sales = np.random.binomial(n=self.buyers_per_round, p=prob_buy)

        reward = (p_t - self.cost) * num_of_sales
        
        self.current_round += 1 # Advance the environment's internal clock
        return num_of_sales, reward

In [ ]:
class PricingEnvironmentSingleProductDriftingMean(Environment):
    def __init__(self, cost, initial_mean_valuation, std_valuation, buyers_per_round, drift_rate):
        self.cost = cost
        self.initial_mean_valuation = initial_mean_valuation
        self.std_valuation = std_valuation
        self.buyers_per_round = buyers_per_round
        self.drift_rate = drift_rate            # How much the mean valuation changes per round
        self.current_round = 0                  # To track time

    def round(self, p_t):
        current_mean_valuation = self.initial_mean_valuation + (self.current_round * self.drift_rate)

        # Probability of a single customer buying at price p_t
        # loc=current_mean_valuation means the center of the valuation distribution shifts
        prob_buy = 1 - norm.cdf(p_t, loc=current_mean_valuation, scale=self.std_valuation)

        # Ensure probability is within valid range [0, 1] (important if drift leads to very high/low valuations)
        prob_buy = np.clip(prob_buy, 0, 1)

        # Number of sales is drawn from a binomial distribution
        num_of_sales = np.random.binomial(n=self.buyers_per_round, p=prob_buy)

        reward = (p_t - self.cost) * num_of_sales
        
        self.current_round += 1 # Advance the environment's internal clock
        return num_of_sales, reward

### Now define the setting...

In [ ]:
# --- Global Settings ---

N = 1                     # Number of products (unused in this version)
T = 20000                 # Time horizon (number of rounds)
n_trials = 10             # Number of independent trials for averaging

# --- Pricing Grid and Product Info ---

cost = 2.00                       # Unit production cost
value = 3.00                      # True mean customer valuation
std_valuation = 1.0               # Standard deviation of customer valuations

min_p = 0.0
max_p = int(value * 2)
price_step = 0.5
P = np.linspace(min_p, max_p, int((max_p - min_p) / price_step) + 1)    # Discrete set of prices
K = len(P)                                                              # Number of price options (arms)

assert cost < value , "Cost must be less than value"

# --- Market and Inventory Parameters ---

B = 120000 * N                          # Total inventory available (seller capacity)

# --- Derived Buyer Generation per Round ---

buyers_per_round = 50                   # Number of buyers per round per product
assert buyers_per_round >= 1, "Must have at least one buyer per round"

# --- Other Parameters ---

zoom = 0.001              # Zoom factor for plotting
s = 30                    # random seed